In [1]:
import csv
import pandas as pd
import numpy as np
import math
from datetime import datetime as dt, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None 

from funcs import driver_status


In [2]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is B0D1-7E59

 Directory of C:\Users\hp\Desktop\due_dilligence_2

11/27/2022  04:14 PM    <DIR>          .
11/27/2022  04:14 PM    <DIR>          ..
11/27/2022  01:08 PM    <DIR>          .ipynb_checkpoints
11/27/2022  03:14 PM         7,763,232 data21.xlsx
11/27/2022  03:20 PM        14,439,045 data22.xlsx
11/27/2022  03:21 PM         1,524,300 datasep.xlsx
11/27/2022  04:14 PM            14,463 due_dilligence2_(driver_analysis).ipynb
11/27/2022  04:14 PM             1,481 funcs.py
11/27/2022  01:06 PM               734 Read Me.txt
11/27/2022  04:15 PM    <DIR>          __pycache__
               6 File(s)     23,743,255 bytes
               4 Dir(s)  19,714,174,976 bytes free


I had to break down the data from the question raw data in illa overview to 3 questions using illa overview's filters for 2021,2022 and sep2022 to download the 3 excels and use then merge them here to have all the transactional data in one df 

In [3]:
df1 = pd.read_excel("data21.xlsx")
df2 = pd.read_excel("data22.xlsx")
df3 = pd.read_excel("datasep.xlsx")

In [4]:
df = pd.concat([df1,df2], axis = 0)
df = pd.concat([df,df3], axis = 0)
df["trip_date"]= pd.to_datetime(df["trip_date"])
df["month_year"] = df["trip_date"].dt.strftime('%m/%Y').astype('datetime64')

In [5]:
print(df["trip_date"].min())
print(df["trip_date"].max())

2021-06-01 00:00:00
2022-09-30 00:00:00


##### Slide2 __(active vs new drivers)

In [19]:
########### We aggregate data according to month year  
driver_month_pivot =  df.pivot_table(index=["driver_name"], 
                                    columns=["month_year"], 
                                    values=["id"], 
                                    aggfunc="count", 
                                    fill_value=0)
driver_month_pivot = driver_month_pivot.applymap(lambda x: 1 if x>0 else 0)
########## Func driver status is imported from the module funcs in the same direcotry with two arguments
########## df and number of months(cols), it just shows the status of each driver everymonth 


driver_month_pivot= driver_month_pivot.reset_index()
cols  = [str(i[1])[:-9] for i in list(driver_month_pivot.columns)]
cols.insert(1,"driver")
cols= cols[1:]
driver_month_pivot.columns = cols
driver_month_pivot= driver_month_pivot.set_index("driver")
driver_month_status = pd.DataFrame(driver_month_pivot.apply(lambda x: pd.Series(driver_status(x,len(driver_month_pivot.columns))), axis=1))

driver_month_status.columns = driver_month_pivot.columns

driver_month_status
month_status_pivot = pd.DataFrame(driver_month_status.replace("unreg", np.NaN).apply(lambda x: pd.value_counts(x)))
month_status_pivot = month_status_pivot.fillna(0)
month_status_pivot 

,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01
active,0.0,124,170,182,215,221,216,262,285,293,374,340,342,359,346,297
inactive,0.0,134,286,440,562,689,914,1127,1298,1446,1714,2018,2242,2509,2749,2952
new,258.0,198,166,155,133,220,259,194,156,349,270,226,284,227,154,197
